In [3]:

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import pathlib
import os


In [6]:

# Path dataset lokal
base_dir = pathlib.Path("../raw_data")
batch_size = 32
img_height = 150
img_width = 150


In [7]:

train_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)


Found 810 files belonging to 3 classes.
Using 648 files for training.
Found 810 files belonging to 3 classes.
Using 162 files for validation.


In [8]:

normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds   = val_ds.map(lambda x, y: (normalization_layer(x), y))

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])


In [9]:

from tensorflow.keras import Input

model = models.Sequential([
    Input(shape=(150, 150, 3)),
    data_augmentation,
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(256, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(512, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])


In [10]:

early_stop = callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

target_acc = callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=2,
    mode='max',
    baseline=0.99,
    restore_best_weights=True
)

callbacks_list = [early_stop, target_acc]

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [11]:

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=callbacks_list
)


Epoch 1/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 71s 3s/step - accuracy: 0.3657 - loss: 1.4652 - val_accuracy: 0.2593 - val_loss: 1.0920
Epoch 2/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.5015 - loss: 0.9079 - val_accuracy: 0.6667 - val_loss: 0.6663
Epoch 3/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.6420 - loss: 0.6772 - val_accuracy: 0.8025 - val_loss: 0.4829
Epoch 4/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.8272 - loss: 0.3970 - val_accuracy: 0.9506 - val_loss: 0.1112
Epoch 5/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 0.9336 - loss: 0.1695 - val_accuracy: 0.9815 - val_loss: 0.0454
Epoch 6/30
21/21 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.9151 - loss: 0.2024 - val_accuracy: 0.8395 - val_loss: 0.4203


In [15]:

output_model_path = pathlib.Path("../model/model.keras")
os.makedirs(output_model_path.parent, exist_ok=True)
model.save(output_model_path)
print(f"✅ Model berhasil disimpan ke: {output_model_path}")


✅ Model berhasil disimpan ke: ..\model\model.keras
